In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/Shareddrives/SI630'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/Shareddrives/SI630


In [ ]:
# !pip install simpletransformers

In [ ]:
import os
from datetime import datetime
import logging
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs


In [ ]:
# Google Data
train_df = pd.read_csv("data/train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("data/dev.tsv", sep="\t").astype(str)
test_df = pd.read_csv("data/test.tsv", sep="\t").astype(str)

train_df = train_df.loc[train_df["label"] == "1"]
eval_df = eval_df.loc[eval_df["label"] == "1"]
test_df = test_df.loc[test_df["label"] == "1"]

train_df = train_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)
test_df = test_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]
test_df = test_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
eval_df["prefix"] = "paraphrase"
test_df["prefix"] = "paraphrase"

# Tuning Parameters

In [ ]:
# # uncomment when tuning parameters
# train_df=train_df[:500]
# eval_df=eval_df[::50]

In [ ]:
# lr_list = [5e-8, 5e-6, 5e-5 5e-4]
# lr_list = [5e-6, 5e-5, 5e-4]
lr_list = [5e-5]

# eps_list = [1e-8, 1e-6]
eps_list = [1e-8]
stats = []

for lr in lr_list:
  for eps in eps_list:

    model_args = {
        "do_sample":True,
        "eval_batch_size":8,
        "evaluate_during_training": True,
        "gradient_accumulation_steps": 4,
        "mlm": True,
        "output_dir": f"outputs/bart-fine-tuned-params/",
        "evaluate_during_training_steps":2500,
        "adam_epsilon": eps,
        "evaluate_during_training_verbose" :True,
        "fp16" : False,
        "learning_rate" : lr,
        "max_length" : 256,
        "max_seq_length" : 256,
        "num_beams" : None,
        "num_return_sequences" : 3,
        "num_train_epochs" : 2,
        "overwrite_output_dir" : True,
        "reprocess_input_data" : True,
        "save_eval_checkpoints" : False,
        "save_steps" : -1,
        "top_k" : 50,
        "top_p" : 0.95,
        "train_batch_size" : 4,
        "use_multiprocessing" : False,
        "wandb_project" : "Paraphrasing with BART",
        'repetition_penalty':3.0,
    }

    model = Seq2SeqModel(encoder_decoder_type="bart", encoder_decoder_name="facebook/bart-large", args=model_args,)


    # training

    
    model.train_model(train_df, eval_data=eval_df)
    eval_loss=model.results['eval_loss']
    eval_perplexity=math.exp(eval_loss)
    stats.append({'learning_rate':lr,\
                  'adam_epsilon':eps,\
                  'eval_loss':eval_loss,\
                  'eval_perplexity':eval_perplexity})

# Training

In [ ]:
model_args = {
    "do_sample":True,
    "eval_batch_size":4,
    "evaluate_during_training": True,
    "gradient_accumulation_steps": 8,
    "mlm": True,
    "output_dir": f"outputs/bart-fine-tuned-penalty/",
    "evaluate_during_training_steps":2500,
    "adam_epsilon": 1e-8,
    "evaluate_during_training_verbose" :True,
    "fp16" : False,
    "learning_rate" : 5e-5,
    "max_length" : 256,
    "max_seq_length" : 256,
    "num_beams" : None,
    "num_return_sequences" : 3,
    "num_train_epochs" : 2,
    "overwrite_output_dir" : True,
    "reprocess_input_data" : True,
    "save_eval_checkpoints" : False,
    "save_steps" : -1,
    "top_k" : 50,
    "top_p" : 0.95,
    "train_batch_size" : 4,
    "use_multiprocessing" : False,
    "wandb_project" : "Paraphrasing with BART",
    "repetition_penalty":3.0,
}

model = Seq2SeqModel(encoder_decoder_type="bart", encoder_decoder_name="facebook/bart-large", args=model_args,)


# training


model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/21829 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


...

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


  0%|          | 0/3539 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/5458 [00:00<?, ?it/s]

  0%|          | 0/3539 [00:00<?, ?it/s]

(1364,
 {'eval_loss': [0.49998122556398145, 0.48277717541671744],
  'global_step': [682, 1364],
  'train_loss': [0.24446943402290344, 0.060692351311445236]})

# Prediction

In [ ]:
model.args

Seq2SeqArgs(adafactor_beta1=None, adafactor_clip_threshold=1.0, adafactor_decay_rate=-0.8, adafactor_eps=(1e-30, 0.001), adafactor_relative_step=True, adafactor_scale_parameter=True, adafactor_warmup_init=True, adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', config={}, cosine_schedule_num_cycles=0.5, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=0, do_lower_case=False, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=4, evaluate_during_training=True, evaluate_during_training_silent=True, evaluate_during_training_steps=2500, evaluate_during_training_verbose=True, evaluate_each_epoch=True, fp16=False, gradient_accumulation_steps=8, learning_rate=5e-05, local_rank=-1, logging_steps=50, manual_seed=None, max_grad_norm=1.0, max_seq_length=256, model_name='facebo

In [ ]:
import logging

from simpletransformers.seq2seq import Seq2SeqModel

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)
# model_args={
#         'do_sample':True,
#         'max_length':256,
#         'top_k':50,
#         'top_p':0.95,
#         'early_stopping':True,
#         'repetition_penalty':3.0,
#         'num_return_sequences':2
# }
model = Seq2SeqModel(
    encoder_decoder_type="bart", encoder_decoder_name="outputs/bart-fine-tuned-penalty",args=model_args
)

In [ ]:
import spacy_universal_sentence_encoder
nlp=spacy_universal_sentence_encoder.load_model('en_use_lg')
def USE(s1, s2):
  s1=nlp(s1)
  s2=nlp(s2)
  similarity = s1.similarity(s2) 
  return similarity
use_lst=[]
s1_lst=test_df['input_text'].tolist()[0:100]
s2_lst=test_df['target_text'].tolist()[0:100]

para_lst = model.predict(s1_lst)

for s1,s2 in zip(s2_lst,para_lst):
  s2=s2[0]
  # print("Targeted:", s1)
  # print("Paraphrased:", s2)
  use_lst.append(USE(s1,s2))

use_score= sum(use_lst)/len(use_lst)
print(use_score)

Generating outputs:   0%|          | 0/25 [00:00<?, ?it/s]

0.9635859253222052


 # Human Evaluation

In [ ]:
joke1= ['A kid finds a magical lamp.He rubs the lamp, and a genie appears and says, “What is your first wish?”',
        ' The kid says, “I wish I were rich!” The genie replies, “It is done! What is your second wish, Rich?”.']
# para1=""
# para2=""
def paraphrase(joke):
  para = model.predict(joke)
  for i in range(len(joke)):
    print("Original:",joke[i])
    print("Paraphrased: ",para[i][0])
paraphrase(joke1)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Original: A kid finds a magical lamp.He rubs the lamp, and a genie appears and says, “What is your first wish?”
Paraphrased:  A kid finds a magical lamp and rubs it. A genie appears and says, `` What is your first wish? ''
Original:  The kid says, “I wish I were rich!” The genie replies, “It is done! What is your second wish, Rich?”.
Paraphrased:  The kid says, “I wish I were rich!” The genie replies, `` It is done! What is your second wish Rich?


In [ ]:
joke2 = ['Three friends stranded on a deserted island find a magic lamp. Inside it is a genie who agrees to grant each friend one wish.',
         ' “I want to go home,” says the first friend. The genie grants her wish.',
         ' “I want to go home, too,” says the second friend. And the genie sends her back home.',
         ' “I’m lonely,” says the third friend. “I sure wish my friends were back here."']
paraphrase(joke2)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Original: Three friends stranded on a deserted island find a magic lamp. Inside it is a genie who agrees to grant each friend one wish.
Paraphrased:  Three friends stranded on a deserted island find a magic lamp inside which is a genie who agrees to grant each friend one wish.
Original:  “I want to go home,” says the first friend. The genie grants her wish.
Paraphrased:  The first friend says “I want to go home,” and the genie grants her wish.
Original:  “I want to go home, too,” says the second friend. And the genie sends her back home.
Paraphrased:  The second friend says “I want to go home too,” and the genie sends her back home.
Original:  “I’m lonely,” says the third friend. “I sure wish my friends were back here."
Paraphrased:  The third friend says, “I’m lonely. I sure wish my friends were back here.


In [ ]:
joke3=['A man walks into a library, approaches the librarian and says, “I’ll have a cheeseburger and fries, please.”',
       'The librarian says, “Sir, you know you’re in a library, right?”',
       '“Sorry,” he whispers. “I’ll have a cheeseburger and fries, please.”']
paraphrase(joke3)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Original: A man walks into a library, approaches the librarian and says, “I’ll have a cheeseburger and fries, please.”
Paraphrased:  A man walks into a library, approaches the librarian and says, “I’ll have a cheeseburger and fries please.
Original: The librarian says, “Sir, you know you’re in a library, right?”
Paraphrased:  The librarian says to him, “Sir, you know you are in a library right?
Original: “Sorry,” he whispers. “I’ll have a cheeseburger and fries, please.”
Paraphrased:  He whispers. “I’ll have a cheeseburger and fries, please.
